# SHREG reduction notebook
This is a template for the SHREG data reduction notbook, which in the end should exist for every ATCA observing project in the SHREG project. For this notebook to work, one has to first find out and set:
 - the name of the ATCA observing project (C*)
 - the dates of observations
 - the name of the science sources
 - the names of the bandpass and phase calibrator

It also has to be indicated whether data:
 - is organised in a folder structur like [project]/[science_source_name]/[obs_date] (useful if only one science source was observed per observation, e.g. C217) or [project]/[obs_date] (useful if multiple science sources were observed per observation, e.g. C942). 
 - should be averaged and how to do so.
 - whether data was taken with CABB or the pre-2008 correlator. 

## Import relevant packages

In [ ]:
import data_reduction_modules as drm
from mirpy import miriad
import os
import glob
import shutil
import subprocess32
import sys
import numpy as np
import logging
from datetime import datetime

In [ ]:
project = 'C819'

In [ ]:
log_logging = datetime.now().strftime(
    "LogFiles/{}_%Y-%m-%d_logging.log".format{project})
log_notes = open(datetime.now().strftime(
    "LogFiles/{}_%Y-%m-%d_notes.log".format{project}), 'a+')
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', 
                    level=logging.WARNING, datefmt='%I:%M:%S', 
                    filename = log_logging)
logging.captureWarnings(True)

## Set names and other input

In [ ]:
# What to do? Only those tasks with a True flag are executed. 
do_read_in = True
do_averaging = False
do_flagging = False
do_calibration = False
do_show_calibresult = False
do_apply_calib = False
do_cont_sub = False
do_imaging = False

In [ ]:
base_folder = '/data/HI_Interferometry/'
# Which correlator was used?
cabb_flag = False
pre08_flag = True
# Should the data be averaged?
uvaver_flag = False
# The following flag indicates whether we have already run the calibration
# once (False) or whether we need to run the calibration steps again 
# because we were not satisfied with the previous solutions (True). If the
# latter is the case, we run blflag before running the calibration step. 
redo_calib_flag = False
# How many channels do the data have? You can select from 512, 1024, 
# 2048 (CABB, no zooms) or 17000 (CABB concatanated zooms), this is impor-
# tant to use the right channel range to subtract the continuum emission 
# from the spectral line observations. 
num_chans = 512
# Initially only put the dates, which are the keys of the obs_dates dictionary
# then let atlod and uvsplit run, clean up each date folder and then set 
# the names of the phase and bandpass calibrators as well as the science 
# sources. Note that there can be multiple science sources. If there are 
# also multiple phase and/or bandpass calibrators, then the list of cali-
# brators needs to be organised such that the calibrators can be used to 
# calibrate the science sources at the same positions in their respective
# lists, i.e. calibrator 0 is used for science source 0, even if that means
# that one calibrator is mentioned multiple times. If only one calibator is
# given it is used to calibrate the data of all science sources. 
obs_dates ={'': {'phase': '', 
                           'bandpass': '', 
                           'sci_sou':  ['', '', 
                                       '', '']},
            '': {'phase': '', 
                             'bandpass': '', 
                             'sci_sou':  ['', '', 
                                         '', '']},
            '': {'phase': '', 
                             'bandpass': '', 
                             'sci_sou':  ['', '', 
                                         '', '']},
            '': {'phase': '', 
                             'bandpass': '', 
                             'sci_sou':  ['', '', 
                                         '', '']}
           }
            
obs_dates_order = ['', '', '', 
                   '']
# This is just a list of dates for which, we run the calibration
dates_do_calib = []
# This is just a list of objects, for which the data reduction is
# completed
objects_finished = []

## Read in the data and split into single source observations

In [ ]:
# Once the data has been read in, split and then cleaned by hand, this step
# will not work anymore because some especially observations where 
# galaxies at different frequencies have been observed alternatly these
# observations will have been moved into different folders. 
if do_read_in == True:
    for date in obs_dates.keys():
        print(date)
        directory = '{}{}/{}'.format(base_folder, project, date)
        log_notes = drm.read_in_data(directory, project, log_notes, 
                                     ifsel=1, ifsel_flag=True)
        log_notes = drm.split_data(directory, log_notes)

## Average the data

In [ ]:
if do_averaging == True:
    if uvaver_flag == True:
        print('I should be averaging data, but I am doing nothing.')

## Flag data
In this section we first flag the amplitude and phase of the bandpass and phase calibrator manually on a baseline by baseline bases using `blflag`. Then we let `uvflag` and `pgflag` do the rest automatically. 

In [ ]:
if do_flagging == True:
    for date in obs_dates_order:
        print(date)
        directory = '{}{}/{}'.format(base_folder, project, date)
        log_notes = drm.blflag_data(directory, obs_dates[date]['bandpass'], 
                                    obs_dates[date]['phase'], log_notes)
    for date in obs_dates_order:
        print(date)
        directory = '{}{}/{}'.format(base_folder, project, date)
        source_list = []
        source_list = source_list + obs_dates[date]['sci_sou']
        source_list = source_list + [obs_dates[date]['phase']]
        source_list = source_list + [obs_dates[date]['bandpass']]
        log_notes = drm.uvflag_data(directory, source_list, log_notes)
        log_notes = drm.pgflag_data(directory, source_list, log_notes)

In [ ]:
if do_flagging == True:
    for date in obs_dates_order:
        for source in obs_dates[date]['sci_sou']:
            print('pgflag vis={}/{} device=/xs mode=amplitude'.format(date, source))

## Calibration
Calibration is always performed for all dates listed in `dates_do_calib`. The general approach would be to put all dates into this list, i.e. perform the calibration for all dates. Then set the `redo_calib_flag` to True and remove all those dates from the `dates_do_calib` for which we are happy with the calibration result and run the calibration again. Since the `redo_calib_flag` is now set to True, an interactive flagging routine is run before the calibration is tried again. 

In [ ]:
if do_calibration == True:
    print('Let\'s do the calibration!')
    if pre08_flag == True:
        for date in dates_do_calib:
            print(date)
            directory = '{}{}/{}'.format(base_folder, project, date)
            if (redo_calib_flag == True):
                log_notes = drm.remove_calib_tables(directory, 
                                                    obs_dates[date]['bandpass'], 
                                                    log_notes) 
                log_notes = drm.remove_calib_tables(directory, 
                                                    obs_dates[date]['phase'], 
                                                    log_notes)
                log_notes = drm.reflag_after_calib(directory, 
                                                   obs_dates[date]['bandpass'], 
                                                   log_notes,
                                                   nobase_flag=False)
                log_notes = drm.reflag_after_calib(directory, 
                                                   obs_dates[date]['phase'], 
                                                   log_notes,
                                                   nobase_flag=False)
            log_notes = drm.bandpass_calib_pre08(directory, 
                                                 obs_dates[date]['bandpass'], 
                                                 log_notes)
            log_notes = drm.apply_bandpass_calib(directory, 
                                                 obs_dates[date]['bandpass'], 
                                                 obs_dates[date]['phase'], 
                                                 log_notes)
            log_notes = drm.phase_calib_pre08(directory, 
                                              obs_dates[date]['bandpass'], 
                                              obs_dates[date]['phase'], 
                                              log_notes)
            drm.plot_calibration_result(directory, obs_dates[date]['bandpass'], 
                                        obs_dates[date]['phase'])
    elif cabb_flag == True:
        for date in dates_do_calib:
            print(date)
            directory = '{}{}/{}'.format(base_folder, project, date)
            if (redo_calib_flag == True):
                log_notes = drm.remove_calib_tables(directory, 
                                                    obs_dates[date]['bandpass'], 
                                                    log_notes) 
                log_notes = drm.remove_calib_tables(directory, 
                                                    obs_dates[date]['phase'], 
                                                    log_notes)
                log_notes = drm.reflag_after_calib(directory, 
                                                   obs_dates[date]['bandpass'], 
                                                   log_notes)
                log_notes = drm.reflag_after_calib(directory, 
                                                   obs_dates[date]['phase'], 
                                                   log_notes)
            log_notes = drm.bandpass_calib_cabb(directory, 
                                                obs_dates[date]['bandpass'], 
                                                log_notes)
            log_notes =  drm.apply_bandpass_calib(directory, 
                                                  obs_dates[date]['bandpass'], 
                                                  obs_dates[date]['phase'], 
                                                  log_notes)
            log_notes = drm.phase_calib_cabb(directory, 
                                             obs_dates[date]['bandpass'], 
                                             obs_dates[date]['phase'], 
                                             log_notes)
            drm.plot_calibration_result(directory, obs_dates[date]['bandpass'], 
                                        obs_dates[date]['phase'], 
                                        cabb_flag=cabb_flag)

The following cell opens the quality assesment plots created by the calibration functions with evince from the commandline. 

In [ ]:
if do_show_calibresult == True:
    for date in dates_do_calib:
        print(date)
        directory = '{}{}/{}'.format(base_folder, project, date)
        for f in glob.glob('{}/*.eps'.format(directory)):
            subprocess32.call(['evince', f])

## Apply calibration to science source and subtract continuum

In [ ]:
if do_apply_calib == True:
    for date in obs_dates_order:
        print(date)
        directory = '{}{}/{}'.format(base_folder, project, date)
        for galaxy in obs_dates[date]['sci_sou']:
            log_notes = drm.apply_phase_calib(directory, obs_dates[date]['phase'], 
                                              galaxy, log_notes)

In [ ]:
if do_cont_sub == True:
    for date in obs_dates_order:
        print(date)
        directory = '{}{}/{}'.format(base_folder, project, date)
        for galaxy in obs_dates[date]['sci_sou']:
            log_notes = drm.subtract_continuum(directory, galaxy, num_chans, 
                                               log_notes)

## Get cubes and moment maps

In [ ]:
if do_imaging == True:
    obs_collections = drm.sort_data(obs_dates)
    done_obj = []
    project_directory = '{}{}'.format(base_folder, project)
    print(len(obs_collections.keys()), len(done_obj))
    for gal in obs_collections.keys():
        if gal in objects_finished: continue
        print(gal)
        name_root, rms, ra, dec, log_notes = drm.invert_data(project_directory, gal, 
                                                              obs_collections[gal],
                                                              log_notes,
                                                              weight='rob2', 
                                                              mosaic_flag=False)
        if rms == -99.9:
            print('Invert failed, I will move on to the next galaxy.')
            continue
        log_notes = drm.clean_data(name_root, rms, log_notes, mosaic_flag=False)
        log_notes, success = drm.restor_data(name_root, log_notes, mosaic_flag=False)
        if success == True:
            log_notes = drm.make_moment_maps(name_root, rms, ra, dec, log_notes)
        else:
            print('I will stop here and move on to the next galaxy.')